# **Task 1.1 - Download Files Scipt**
Recieve 10-K Files from Intel Corporation and Microsoft Corp

In [5]:
import os

companies = ["INTC", "MSFT"]
subdirectories = []

In [2]:
from sec_edgar_downloader import Downloader
from bs4 import BeautifulSoup
dl = Downloader("Company Name", "my.email@domain.com")
for company in companies:
    dl.get("10-K", company, after = "1995-01-01", before = "2024-01-01", download_details = True)
    org_path = "sec-edgar-filings/"+company+"/10-K/"

    for fileName in os.listdir(org_path):
        if fileName.startswith("000"):
            newName = fileName[11:]
            if (newName[0] == '9'):
                newName = "19" + newName 
            else:
                newName = "20" + newName

            path = org_path + fileName
            path_list = os.listdir(path)
            new_path= org_path + newName

            if (os.path.exists(new_path)):
                path_dir = os.listdir(new_path)
                if ((newName[2] == "9" or newName[2] == "0") and len(path_dir) == 2):
                    if (os.path.exists(new_path+"/"+path_list[1])):
                        os.remove(new_path+"/"+path_list[1])
                    elif (os.path.exists(new_path+"/"+path_list[0])):
                        os.remove(new_path+"/"+path_list[0])
                if (len(path_list) == 2):
                    os.remove(path+"/"+path_list[1])
                os.remove(path+"/"+path_list[0])
                os.rmdir(path)
            else:
                os.rename(path, new_path)
                if (len(path_list) == 2):
                    if (newName[2] == "9" or newName[2] == "0"):
                        os.remove(new_path+"/"+path_list[1])
                    else:
                        os.remove(new_path+"/"+path_list[0])

    temp = [d for d in os.listdir(org_path) if os.path.isdir(os.path.join(org_path, d))]
    for i in range(len(temp)):
        subdirectories.append(temp[i])

ModuleNotFoundError: No module named 'sec_edgar_downloader'

## **Make Files Readable for Interpretation**

In [ ]:
subdirs = subdirectories
companyNum = 0
year = 1995
years = []
years.append(year)
output_path = "10-K Files"

for index, subdirectory in enumerate(subdirs):
    if (index % 29 == 0 and index != 0):
        companyNum = companyNum + 1
    
    path = "sec-edgar-filings/"+companies[companyNum]+"/10-K/"+subdirectory+"/"
    curr_dir = os.listdir(path)
    path = path+curr_dir[0]
    
    # Open and read the content of file
    with open(path, 'r', encoding='utf-8') as file:
        filing_content = file.read()

    soup = BeautifulSoup(filing_content, 'html.parser')

    # Extract the entire text content
    entire_content = soup.get_text()

    #Save the content to a text file with a unique name
    output_file_name = f"\\10k_{year}.txt"  # Unique name based on index

    if (year + 1 < 2024):
        year += 1
        if (year not in years):
            years.append(year)
    else:
        year = 1995

    output_path = "10-K Files"
    if not os.path.exists(output_path):
        os.mkdir(output_path)

    company_subdir = os.path.join(output_path, companies[companyNum])
    if not os.path.exists(company_subdir):
        os.mkdir(company_subdir)

    output_path = company_subdir
    output_file_path = output_path + output_file_name
    
    with open(output_file_path, 'w', encoding='utf-8') as out_file:
        out_file.write(entire_content)

# **Task 1.2**

In [15]:
import re
def process_line_with_llm(line):
    # Perform LLM inference to extract relevant information from the line
    # For demonstration, let's extract any numeric values from the line
    numeric_values = re.findall(r'\b\d+(?:\.\d+)?\b', line)
    return numeric_values

def is_phrase_in_line(phrase, line):
    return phrase.lower() in line.lower()

In [26]:
netRev = {}
input_dirs = []

for company in companies:
    path = "10-K Files/"+ company    
    curr_dir = os.listdir(path)
    for i in range(len(curr_dir)):
        text_path = path + "/" + curr_dir[i]
        os.path.abspath(text_path)
        if (os.path.exists(text_path)):
            input_dirs.append(text_path)
    netRev[company] = []
    
print(netRev)

10-K Files/INTC/10k_1995.txt
['10-K Files/INTC/10k_1995.txt']
10-K Files/INTC/10k_1996.txt
['10-K Files/INTC/10k_1995.txt', '10-K Files/INTC/10k_1996.txt']
10-K Files/INTC/10k_1997.txt
['10-K Files/INTC/10k_1995.txt', '10-K Files/INTC/10k_1996.txt', '10-K Files/INTC/10k_1997.txt']
10-K Files/INTC/10k_1998.txt
['10-K Files/INTC/10k_1995.txt', '10-K Files/INTC/10k_1996.txt', '10-K Files/INTC/10k_1997.txt', '10-K Files/INTC/10k_1998.txt']
10-K Files/INTC/10k_1999.txt
['10-K Files/INTC/10k_1995.txt', '10-K Files/INTC/10k_1996.txt', '10-K Files/INTC/10k_1997.txt', '10-K Files/INTC/10k_1998.txt', '10-K Files/INTC/10k_1999.txt']
10-K Files/INTC/10k_2000.txt
['10-K Files/INTC/10k_1995.txt', '10-K Files/INTC/10k_1996.txt', '10-K Files/INTC/10k_1997.txt', '10-K Files/INTC/10k_1998.txt', '10-K Files/INTC/10k_1999.txt', '10-K Files/INTC/10k_2000.txt']
10-K Files/INTC/10k_2001.txt
['10-K Files/INTC/10k_1995.txt', '10-K Files/INTC/10k_1996.txt', '10-K Files/INTC/10k_1997.txt', '10-K Files/INTC/10k_1

Looking for the factors Net Revenue

Keywords to Search for is "(In Millions, Except Per Share Amounts)"

Look for the first occurence of a dollar sign after finding all keywords and factors

In [37]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

factorOne = "Net Revenue $"
factorTwo = "Net Revenues"

device = "cuda" if torch.cuda.is_available() else "cpu"

tokenizer = AutoTokenizer.from_pretrained("facebook/opt-1.3b")
# print("Tokenizer complete")
model = AutoModelForCausalLM.from_pretrained("facebook/opt-1.3b")
model.to(device)
# print("Model Initialization Complete")

for i in range(len(input_dirs)):
    file_path = input_dirs[i]
    print(file_path)
    with open(file_path, "r") as file:
        for line in file:
            # Process each line with LLM inference
            line = line.strip()  # Remove leading/trailing whitespace
            print(line)
            inputs = tokenizer(line, return_tensors="pt").to(device)
            outputs = model(**inputs)
            text_generated = tokenizer.decode(outputs[0][0][0][0], skip_special_tokens=True)
            
            if (factorOne.strip() in text_generated.lower() or factorTwo in text_generated):
            # Process the generated text (for demonstration, let's extract numeric values)
                extracted_values = process_line_with_llm(text_generated)
                print(extracted_values)
            # Store found elements in the array
                if input_dirs[11:15] == companies[0]:
                    netRev[companies[0]].append(extracted_values)
                else:
                    netRev[companies[1]].append(extracted_values)

10-K Files/INTC/10k_1995.txt
-----BEGIN PRIVACY-ENHANCED MESSAGE-----


OverflowError: can't convert negative int to unsigned

In [19]:
print(netRev)

{'INTC': [], 'MSFT': []}


# **Task 1.3**

In [36]:
from dash import Dash, html, dcc, callback, dash_table, Output, Input, State
import plotly.express as px
import dash_bootstrap_components as dbc

app = Dash(__name__)

colors = {
    'text': '#000000'
}    

app.layout = html.Div(children=[
    html.H1(
        children='Gestational Diabetes Drug-Drug Interactions',
        style={
            'textAlign': 'center',
            'color': colors['text']
        }
    ),

    html.Div([
        dcc.Dropdown(
            companies, 
            placeholder = "Select a Company",
            id='companyDrop'
        ),

    html.Div(dcc.Graph(id='revenue'))
    ])
])

# @callback(
#     Output('revenue', 'figure'),
#     Input('companyID', 'value')
# )

# def updateMedicationGraph(currInteraction):
#     temp = df.loc[df['Observed Drug-Drug Interaction'] == currInteraction]
#     data = temp['Suspect Product Names']
#     fig = px.histogram(data, x = "Suspect Product Names", title = 'Cases of Patients that Used Drugs')
#     return fig

if __name__ == '__main__':
    app.run(port=3340, jupyter_mode="external")
    # app.run(port = 3340, jupyter_server_url="https://tech-company-10K-explored", jupyter_mode="external")

ModuleNotFoundError: No module named 'dash'